<a href="https://colab.research.google.com/github/i-SanMartin/FastAI/blob/main/04_MNIST_Basics/Network_Exercise_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook fastai==2.2.5
import fastbook

     |████████████████████████████████| 727kB 16.3MB/s 
     |████████████████████████████████| 194kB 46.4MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 1.2MB 54.3MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 12.8MB 242kB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
Path.BASE_PATH = path

In [5]:
categories = tuple(x for x in range(0,10))
categories

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [18]:
#Training data
train_x = torch.tensor([])
train_y = torch.tensor([])

for x in categories:
  number_imgs = (path/'training'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  train_x = torch.cat([train_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  train_y = torch.cat([train_y,y])

In [19]:
#Validation data
valid_x = torch.tensor([])
valid_y = torch.tensor([])

for x in categories:
  number_imgs = (path/'testing'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  valid_x = torch.cat([valid_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  valid_y = torch.cat([valid_y,y])

In [20]:
#Reshape data
train_x = train_x.view(-1,28,28).float()/255
valid_x = valid_x.view(-1,28,28).float()/255
train_y = train_y.unsqueeze(1).long()
valid_y = valid_y.unsqueeze(1).long()
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

(torch.Size([60000, 28, 28]),
 torch.Size([60000, 1]),
 torch.Size([10000, 28, 28]),
 torch.Size([10000, 1]))

In [21]:
#Create datasets
batch_size=256
dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=batch_size,shuffle=True)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(dset, batch_size=batch_size,shuffle=True)
dls = DataLoaders(dl, valid_dl)

In [22]:
#Optimizer
class Sgd():
  def __init__(self, parameters, lr):
    self.parameters = list(parameters)
    self.lr = lr
  def step(self):
    for p in self.parameters:
      p.data -= p.grad * self.lr
  def zero_grad(self):
    for p in self.parameters:
      p.grad = None

In [23]:
#Neural network
simple_net = nn.Sequential(
    nn.Conv2d(1, 32, 5, stride=1, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Conv2d(32, 64, 5, stride=1, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2,2),
    nn.Linear(7*7*64,30),
    nn.ReLU(),
    nn.Linear(30,len(categories)),
    nn.Sigmoid()
)

In [43]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5, stride=1, padding=2)
        self.fc1 = nn.Linear(64*7*7, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64*7*7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
  
net = Net()

In [44]:
#Metric function
def batch_accuracy(x,y):
    pred = net.forward(x.view(-1,1,28,28))
    pred_class = torch.tensor([torch.argmax(i) for i in pred])
    correct = (pred_class == y.squeeze(1)).float().sum()
    return correct

In [45]:
#Learner
class learner():
  def __init__(self, Dls, model, optimize, loss_func, metrics):
    self.dls = Dls
    self.model = model
    self.opt = optimize
    self.loss_func = loss_func
    self.metrics = metrics

  def train(self, epochs, lr):
    self.Opt = self.opt(self.model.parameters(), lr)
    metric = self.validate_epoch(valid_dl).item()
    loss = self.loss_func(self.model(valid_x.view(-1,1,28,28)),valid_y.squeeze(1))
    for i in range(epochs):
      print("Epoch: {}   Metric:  {}   Loss:   {:0.4f}".format(i, metric, loss))
      self.train_epoch(dl)
      metric = self.validate_epoch(valid_dl).item()
      loss = self.loss_func(self.model(valid_x),valid_y.squeeze(1))
    print("Epoch: {}   Metric:  {}   Loss:   {:0.4f}".format(i, metric, loss))

  def predict(x):
    pred = torch.argmax(self.model(x))
    return(pred)

  def train_epoch(self, data):
    for dx, dy in data:
      dx = dx.view(256,1,28,28)
      self.calc_grad(dx,dy)
      self.Opt.step()
      self.Opt.zero_grad()

  def validate_epoch(self, data):
    accs = torch.tensor([0.])
    for xb,yb in data:
      accs += self.metrics(xb,yb)
    return accs/len(data.dataset)

  def calc_grad(self,x,y): 
    pred = self.model(x)
    loss = self.loss_func(pred,y.squeeze(1))
    loss.backward()
    return loss
  

In [46]:
learn = learner(dls, net, Sgd, nn.CrossEntropyLoss(), batch_accuracy)

In [47]:
#Intentar conseguir que llegue a una metrica de 0.8 en 20 
learn.train(50, 0.1)

Streaming output truncated to the last 5000 lines.
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 

RuntimeError: ignored

In [41]:
net.forward(train_x[20000].view(-1,1,28,28))

RuntimeError: ignored